In [4]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pymatgen.core.composition import Composition
import scipy.stats
from scipy.stats import pearsonr
import collections
sns.set_context('paper', font_scale=2)
lgdcnn_dir = r"D:\deep\LGDCNN"

In [5]:
def JS_divergence(p,q):
    M=(p+q)/2
    return 0.5*scipy.stats.entropy(p, M, base=2)+0.5*scipy.stats.entropy(q, M, base=2)

def KL_divergence(p,q):
    return scipy.stats.entropy(p, q, base=2)

def eculidDisSim(x,y):
    '''
    欧几里得相似度
    '''
    return np.sqrt(sum(pow(a-b,2) for a,b in zip(x,y)))

def manhattanDisSim(x,y):
    '''
    曼哈顿相似度
    '''
    return sum(abs(a-b) for a,b in zip(x,y))

def cosSim(x,y):
    '''
    余弦相似度
    '''
    tmp=np.sum(x*y)
    non=np.linalg.norm(x)*np.linalg.norm(y)
    return np.round(tmp/float(non),9)

def pearsonrSim(x,y):
    '''
    皮尔森相似度
    '''
    return pearsonr(x,y)[0]

## 2

In [3]:
str_f = 'As'
s_name = '2'
# %%
# data_dir = f'data/embeddings_L-ATT-RES-DCNN-512/ENV/{str_f}/3'
data_dir = f'data/embeddings_L-ATT-RES-DCNN-512/ENV_OQMD_band_train_val/{str_f}/{s_name}'
if isinstance(data_dir, list):
    mat_props = data_dir
elif os.path.isdir(data_dir):
    mat_props = os.listdir(data_dir)
else:
    mat_props = [data_dir]

# %%
name1 = [i.split('.csv')[0] for i in mat_props if i.split('.csv')[0] == 'fraction_fomula']
path_formula = os.path.join(data_dir, f'{name1[0]}.csv')
df_f = pd.read_csv(path_formula)
formu_2 = df_f['formula'].values

row_ind = 2
save_num_2 = np.zeros((len(mat_props)-1, row_ind, row_ind))

for i, mat_prop in enumerate(mat_props):
    if mat_prop.split('.csv')[0] == 'fraction_fomula':
        mat_props.pop(i)
for i, mat_prop in enumerate(mat_props):   
    if os.path.isfile(mat_prop):
        name = os.path.basename(mat_prop).split('.csv')[0]
        path = mat_prop
    elif os.path.isdir(data_dir):
        name = mat_prop.split('.csv')[0]
        path = os.path.join(data_dir, f'{name}.csv')
    else:
        path = data_dir
        name = mat_prop.split('/')[-1].split('.csv')[0]
        
    # read csv files
    df = pd.read_csv(path)
    if 'element' not in df.columns:
        df['element'] = df.iloc[:,0]
        df = df.drop(columns=['Unnamed: 0'])

    # get rid of 'Null' element representation
    null_row = df['element'] == 'Null'
    df = df[~null_row]

    # grab only the elements that are in oliynyk
    df.index = df['element']
    
    # df.index = df['model']
    # df = df.loc[oliynyk_elements]

    elements = df.pop('element')
    
    # %%
    # normalize the dataframe
    # zero mean, unit variance
    # df = (df - df.mean(numeric_only=True)) / df.std(numeric_only=True)
  
    current_elements = elements
    df_extended = df.reindex(current_elements)
    dfT = df_extended.T
    dfT.columns = current_elements
    corr2 = dfT.corr()

    save_num_2[i,:,:] = corr2.values
    # mask = np.triu(np.ones_like(corr2, dtype=bool))



thred_v = 0.01
# thred_v = 0.1
all_dict2 = dict(zip(formu_2, save_num_2)) 
index_coll2 = dict()

k = 0
for k1,values in all_dict2.items():
    ll_k = []
    ll_v = []
    vv_k = []
    vv_v = []
    for k2,value_q in all_dict2.items():
        if np.abs(np.subtract(values, value_q)).max() <= thred_v:
            ll_v.append(value_q)
            ll_k.append(k2)
        else:
            vv_v.append(value_q)
            vv_k.append(k2)
    if len(ll_v) != 0:
        index_coll2[str(k)] = dict(zip(ll_k, ll_v))
        k = k+1
    if len(vv_v) != 0 and len(vv_v) != 1: 
        all_dict2 = dict()
        all_dict2 = dict(zip(vv_k, vv_v))
    else:
        break
        
if len(vv_v) != 0:       
    index_coll2[str(k)] = dict(zip(vv_k, vv_v))

# puzzle large matrix
puzzle_dict = {}
for i,v in index_coll2.items():
    for k, dic_v in v.items():
        puzzle_dict[i] = dict(zip([k],[dic_v]))
        continue

# 得到每个环境下的一个pearson矩阵放到puzzle_arr
puzzle_arr = np.zeros((len(index_coll2), row_ind, row_ind))
for i,v in puzzle_dict.items():
    for _, subsets in v.items():
            puzzle_arr[int(i),:,:] = subsets

# puzzle_arr.shape[0],3,3 转成 puzzle_arr.shape[0],3,2
puzzle_arr_downscale = np.zeros((len(index_coll2), row_ind, row_ind-1))
for i in range(puzzle_arr.shape[0]):
    A = puzzle_arr[i]
    puzzle_arr_downscale[i,:,:] = A[~np.eye(A.shape[0],dtype=bool)].reshape(A.shape[0],-1)

# 降维 reshape 方便绘制heatmap
puzzle_arr_trans = puzzle_arr_downscale.reshape(len(index_coll2),-1) 

# 删除array中的相同值
puzzle_arr_trans_diff2 = np.unique(puzzle_arr_trans, axis=1)
corr_2 = np.linalg.norm(puzzle_arr_trans_diff2, ord=np.Inf, axis=1, keepdims=True) # 范数inf

IndexError: list index out of range

In [ ]:
sorted(corr_2)

[array([0.63856586]),
 array([0.67559784]),
 array([0.70736113]),
 array([0.74886336]),
 array([0.8327496]),
 array([0.84957768]),
 array([0.86919645]),
 array([0.88582961]),
 array([0.90945419]),
 array([0.92500802]),
 array([0.93706058]),
 array([0.95225359]),
 array([0.96438995]),
 array([0.97630477]),
 array([0.98012227]),
 array([0.98032474]),
 array([0.99168302])]

In [ ]:
sorted(corr_2)

[array([0.67559784]),
 array([0.70736113]),
 array([0.74886336]),
 array([0.8327496]),
 array([0.84957768]),
 array([0.90945419]),
 array([0.92500802]),
 array([0.93706058]),
 array([0.96438995])]

In [ ]:
sorted(corr_2)

[array([0.70736113]),
 array([0.74886336]),
 array([0.8327496]),
 array([0.86919645]),
 array([0.88582961]),
 array([0.96438995])]

## 3

In [6]:
str_f = 'Si'
# s_d = 'ENV_OQMD_band_test'
# s_d = 'ENV_aflow_band_test'
# s_d = 'ENV_OQMD_Formation_Enthalpy_test'
# s_d = 'ENV_OQMD_Energy_per_atom_test'
# s_d = 'ENV_aflow__energy_atom_test'
# s_d = 'ENV_matbench_mp_gap_test'
# s_d = 'ENV_matbench_log_gvrh0_test'
# s_d = 'ENV_OQMD_Volume_per_atom_test'
s_d = 'ENV_CritExam_Ef_test'
# dir_s = 'CECV/embeddings_LSTM'
# dir_s = 'CECV/embeddings_DCNN_K1'
# dir_s = 'CECV/embeddings_DCNN'
# dir_s = 'CECV/embeddings_LDCNN' #LDCNN
dir_s = 'CECV/embeddings_L-G-DCNN'    # L-G-DCNN
num = 5
s_n = 'DCNN_sequential_' + str(num) + '_layers'
# s_n = 'TransformerEncoder' + str(num) + '_layers'
# s_n = 'Dropout' + str(num) + '_layers'
# s_n = 'LSTM_output' + str(num)
# s_n = 'GRU_output' + str(num)
# s_n = 'RNN_output' + str(num)
# s_n = 'embeddings_RDCNN' + str(num)

for thred_v in [0.05,0.1,0.15,0.2,0.25,0.3,0.35,0.4]:
    s_name = '3'
    data_dir = os.path.join(lgdcnn_dir,"data", dir_s, s_d, str_f,s_n,s_name)
    if isinstance(data_dir, list):
        mat_props = data_dir
    elif os.path.isdir(data_dir):
        mat_props = os.listdir(data_dir)
    else:
        mat_props = [data_dir]
    # %%
    name1 = [i.split('.csv')[0] for i in mat_props if i.split('.csv')[0] == 'fraction_fomula']
    path_formula = os.path.join(data_dir, f'{name1[0]}.csv')
    df_f = pd.read_csv(path_formula)
    formu = df_f['formula'].values
    row_ind = 3
    save_num = np.zeros((len(mat_props)-1,row_ind,row_ind))
    for i, mat_prop in enumerate(mat_props):
        if mat_prop.split('.csv')[0] == 'fraction_fomula':
            mat_props.pop(i)
    for i, mat_prop in enumerate(mat_props):   
        if os.path.isfile(mat_prop):
            name = os.path.basename(mat_prop).split('.csv')[0]
            path = mat_prop
        elif os.path.isdir(data_dir):
            name = mat_prop.split('.csv')[0]
            path = os.path.join(data_dir, f'{name}.csv')
        else:
            path = data_dir
            name = mat_prop.split('/')[-1].split('.csv')[0]
        # read csv files
        df = pd.read_csv(path)
        if 'element' not in df.columns:
            df['element'] = df.iloc[:,0]
            df = df.drop(columns=['Unnamed: 0'])
        null_row = df['element'] == 'Null'
        df = df[~null_row]
        df.index = df['element']
        elements = df.pop('element')
        df = (df - df.mean(numeric_only=True)) / df.std(numeric_only=True)
        current_elements = elements
        df_extended = df.reindex(current_elements)
        dfT = df_extended.T
        dfT.columns = current_elements
        corr2 = dfT.corr()
        save_num[i,:,:] = corr2.values
    all_dict = dict(zip(formu, save_num)) 
    index_coll = collections.OrderedDict()
    k = 0
    for k1,values in all_dict.items():
        ll_k = []
        ll_v = []
        vv_k = []
        vv_v = []
        for k2,value_q in all_dict.items():
            if np.abs((np.subtract(np.abs(values), np.abs(value_q)))).max() < thred_v:
                ll_v.append(value_q)
                ll_k.append(k2)
            else:
                vv_v.append(value_q)
                vv_k.append(k2)
        if len(ll_v) != 0:
            index_coll[str(k)] = dict(zip(ll_k, ll_v))
            k = k+1
        if len(vv_v) != 0 and len(vv_v) != 1: 
            all_dict = dict()
            all_dict = dict(zip(vv_k, vv_v))
        else:
            break
    if len(vv_v) != 0:       
        index_coll[str(k)] = dict(zip(vv_k, vv_v))
    puzzle_dict_3 = collections.OrderedDict()
    for i,v in index_coll.items():
            first_key = next(iter(v))
            puzzle_dict_3[i] = dict(zip([first_key],[v[first_key]]))
            continue
############################################################################################
    s_name = '4'
    data_dir = os.path.join(lgdcnn_dir,"data", dir_s, s_d, str_f,s_n,s_name)
    if isinstance(data_dir, list):
        mat_props = data_dir
    elif os.path.isdir(data_dir):
        mat_props = os.listdir(data_dir)
    else:
        mat_props = [data_dir]
    name1 = [i.split('.csv')[0] for i in mat_props if i.split('.csv')[0] == 'fraction_fomula']
    path_formula = os.path.join(data_dir, f'{name1[0]}.csv')
    df_f = pd.read_csv(path_formula)
    formu_4 = df_f['formula'].values
    row_ind_4 = 4
    save_num_4 = np.zeros((len(mat_props)-1, row_ind_4, row_ind_4))
    for i, mat_prop in enumerate(mat_props):
        if mat_prop.split('.csv')[0] == 'fraction_fomula':
            mat_props.pop(i)
    for i, mat_prop in enumerate(mat_props):   
        if os.path.isfile(mat_prop):
            name = os.path.basename(mat_prop).split('.csv')[0]
            path = mat_prop
        elif os.path.isdir(data_dir):
            name = mat_prop.split('.csv')[0]
            path = os.path.join(data_dir, f'{name}.csv')
        else:
            path = data_dir
            name = mat_prop.split('/')[-1].split('.csv')[0]
        df = pd.read_csv(path)
        if 'element' not in df.columns:
            df['element'] = df.iloc[:,0]
            df = df.drop(columns=['Unnamed: 0'])
        null_row = df['element'] == 'Null'
        df = df[~null_row]
        df.index = df['element']
        elements = df.pop('element')
        df = (df - df.mean(numeric_only=True)) / df.std(numeric_only=True)
        current_elements = elements
        df_extended = df.reindex(current_elements)
        dfT = df_extended.T
        dfT.columns = current_elements
        corr2 = dfT.corr()
        save_num_4[i,:,:] = corr2.values
    all_dict_4 = dict(zip(formu_4, save_num_4)) 
    index_coll_4 = collections.OrderedDict()
    k = 0
    for k1,values in all_dict_4.items():
        ll_k = []
        ll_v = []
        vv_k = []
        vv_v = []
        for k2,value_q in all_dict_4.items():
            if np.abs(np.subtract(values, value_q)).max() <= thred_v:
                ll_v.append(value_q)
                ll_k.append(k2)
            else:
                vv_v.append(value_q)
                vv_k.append(k2)
        if len(ll_v) != 0:
            index_coll_4[str(k)] = dict(zip(ll_k, ll_v))
            k = k+1
        if len(vv_v) != 0 and len(vv_v) != 1: 
            all_dict_4 = dict()
            all_dict_4 = dict(zip(vv_k, vv_v))
        else:
            break
    if len(vv_v) != 0:       
        index_coll_4[str(k)] = dict(zip(vv_k, vv_v))
    puzzle_dict_4 = collections.OrderedDict()
    for i,v in index_coll_4.items():
            first_key = next(iter(v))
            puzzle_dict_4[i] = dict(zip([first_key],[v[first_key]]))
            continue
#####################################################################################################################
    s_name = '5'
    data_dir = os.path.join(lgdcnn_dir,"data", dir_s, s_d, str_f,s_n,s_name)
    if isinstance(data_dir, list):
        mat_props = data_dir
    elif os.path.isdir(data_dir):
        mat_props = os.listdir(data_dir)
    else:
        mat_props = [data_dir]

    name1 = [i.split('.csv')[0] for i in mat_props if i.split('.csv')[0] == 'fraction_fomula']
    path_formula = os.path.join(data_dir, f'{name1[0]}.csv')
    df_f = pd.read_csv(path_formula)
    formu_5 = df_f['formula'].values

    row_ind_5 = 5
    save_num_5 = np.zeros((len(mat_props)-1, row_ind_5, row_ind_5))

    for i, mat_prop in enumerate(mat_props):
        if mat_prop.split('.csv')[0] == 'fraction_fomula':
            mat_props.pop(i)
    for i, mat_prop in enumerate(mat_props):   
        if os.path.isfile(mat_prop):
            name = os.path.basename(mat_prop).split('.csv')[0]
            path = mat_prop
        elif os.path.isdir(data_dir):
            name = mat_prop.split('.csv')[0]
            path = os.path.join(data_dir, f'{name}.csv')
        else:
            path = data_dir
            name = mat_prop.split('/')[-1].split('.csv')[0]
        df = pd.read_csv(path)
        if 'element' not in df.columns:
            df['element'] = df.iloc[:,0]
            df = df.drop(columns=['Unnamed: 0'])
        null_row = df['element'] == 'Null'
        df = df[~null_row]
        df.index = df['element']
        elements = df.pop('element')
        df = (df - df.mean(numeric_only=True)) / df.std(numeric_only=True)
        current_elements = elements
        df_extended = df.reindex(current_elements)
        dfT = df_extended.T
        dfT.columns = current_elements
        corr2 = dfT.corr()
        save_num_5[i,:,:] = corr2.values
    all_dict_5 = dict(zip(formu_5, save_num_5)) 
    index_coll_5 = collections.OrderedDict()
    k = 0
    for k1,values in all_dict_5.items():
        ll_k = []
        ll_v = []
        vv_k = []
        vv_v = []
        for k2,value_q in all_dict_5.items():
            if np.abs(np.subtract(values, value_q)).max() <= thred_v:
                ll_v.append(value_q)
                ll_k.append(k2)
            else:
                vv_v.append(value_q)
                vv_k.append(k2)
        if len(ll_v) != 0:
            index_coll_5[str(k)] = dict(zip(ll_k, ll_v))
            k = k+1
        if len(vv_v) != 0 and len(vv_v) != 1: 
            all_dict_5 = dict()
            all_dict_5 = dict(zip(vv_k, vv_v))
        else:
            break
    if len(vv_v) != 0:       
        index_coll_5[str(k)] = dict(zip(vv_k, vv_v))
    puzzle_dict_5 = collections.OrderedDict()
    for i,v in index_coll_5.items():
            first_key = next(iter(v))
            puzzle_dict_5[i] = dict(zip([first_key],[v[first_key]]))
            continue
#####################################################################################################################
    s_name = '6'
    data_dir = os.path.join(lgdcnn_dir,"data", dir_s, s_d, str_f,s_n,s_name)
    if isinstance(data_dir, list):
        mat_props = data_dir
    elif os.path.isdir(data_dir):
        mat_props = os.listdir(data_dir)
    else:
        mat_props = [data_dir]
    # %%
    name1 = [i.split('.csv')[0] for i in mat_props if i.split('.csv')[0] == 'fraction_fomula']
    path_formula = os.path.join(data_dir, f'{name1[0]}.csv')
    df_f = pd.read_csv(path_formula)
    formu_6 = df_f['formula'].values
    row_ind_6 = 6
    save_num_6 = np.zeros((len(mat_props)-1, row_ind_6, row_ind_6))
    for i, mat_prop in enumerate(mat_props):
        if mat_prop.split('.csv')[0] == 'fraction_fomula':
            mat_props.pop(i)
    for i, mat_prop in enumerate(mat_props):   
        if os.path.isfile(mat_prop):
            name = os.path.basename(mat_prop).split('.csv')[0]
            path = mat_prop
        elif os.path.isdir(data_dir):
            name = mat_prop.split('.csv')[0]
            path = os.path.join(data_dir, f'{name}.csv')
        else:
            path = data_dir
            name = mat_prop.split('/')[-1].split('.csv')[0]  
        df = pd.read_csv(path)
        if 'element' not in df.columns:
            df['element'] = df.iloc[:,0]
            df = df.drop(columns=['Unnamed: 0'])
        null_row = df['element'] == 'Null'
        df = df[~null_row]
        df.index = df['element']
        elements = df.pop('element')
        df = (df - df.mean(numeric_only=True)) / df.std(numeric_only=True)
        current_elements = elements
        df_extended = df.reindex(current_elements)
        dfT = df_extended.T
        dfT.columns = current_elements
        corr2 = dfT.corr()
        save_num_6[i,:,:] = corr2.values
    all_dict_6 = dict(zip(formu_6, save_num_6)) 
    index_coll_6 = collections.OrderedDict()

    k = 0
    for k1,values in all_dict_6.items():
        ll_k = []
        ll_v = []
        vv_k = []
        vv_v = []
        for k2,value_q in all_dict_6.items():
            if np.abs(np.subtract(values, value_q)).max() <= thred_v:
                ll_v.append(value_q)
                ll_k.append(k2)
            else:
                vv_v.append(value_q)
                vv_k.append(k2)
        if len(ll_v) != 0:
            index_coll_6[str(k)] = dict(zip(ll_k, ll_v))
            k = k+1
        if len(vv_v) != 0 and len(vv_v) != 1: 
            all_dict_6 = dict()
            all_dict_6 = dict(zip(vv_k, vv_v))
        else:
            break
    if len(vv_v) != 0:       
        index_coll_6[str(k)] = dict(zip(vv_k, vv_v))
    puzzle_dict_6 = collections.OrderedDict()
    for i,v in index_coll_6.items():
            first_key = next(iter(v))
            puzzle_dict_6[i] = dict(zip([first_key],[v[first_key]]))
            continue
#####################################################################################################################
    s_name = '7'
    data_dir = os.path.join(lgdcnn_dir,"data", dir_s, s_d, str_f,s_n,s_name)
    if isinstance(data_dir, list):
        mat_props = data_dir
    elif os.path.isdir(data_dir):
        mat_props = os.listdir(data_dir)
    else:
        mat_props = [data_dir]
    # %%
    name1 = [i.split('.csv')[0] for i in mat_props if i.split('.csv')[0] == 'fraction_fomula']
    path_formula = os.path.join(data_dir, f'{name1[0]}.csv')
    df_f = pd.read_csv(path_formula)
    formu_7 = df_f['formula'].values
    row_ind_7 = 7
    save_num_7 = np.zeros((len(mat_props)-1, row_ind_7, row_ind_7))
    for i, mat_prop in enumerate(mat_props):
        if mat_prop.split('.csv')[0] == 'fraction_fomula':
            mat_props.pop(i)
    for i, mat_prop in enumerate(mat_props):   
        if os.path.isfile(mat_prop):
            name = os.path.basename(mat_prop).split('.csv')[0]
            path = mat_prop
        elif os.path.isdir(data_dir):
            name = mat_prop.split('.csv')[0]
            path = os.path.join(data_dir, f'{name}.csv')
        else:
            path = data_dir
            name = mat_prop.split('/')[-1].split('.csv')[0]
        # read csv files
        df = pd.read_csv(path)
        if 'element' not in df.columns:
            df['element'] = df.iloc[:,0]
            df = df.drop(columns=['Unnamed: 0'])
        null_row = df['element'] == 'Null'
        df = df[~null_row]
        df.index = df['element']
        elements = df.pop('element')
        df = (df - df.mean(numeric_only=True)) / df.std(numeric_only=True)
        current_elements = elements
        df_extended = df.reindex(current_elements)
        dfT = df_extended.T
        dfT.columns = current_elements
        corr2 = dfT.corr()
        save_num_7[i,:,:] = corr2.values
    all_dict_7 = dict(zip(formu_7, save_num_7)) 
    index_coll_7 = collections.OrderedDict()
    k = 0
    for k1,values in all_dict_7.items():
        ll_k = []
        ll_v = []
        vv_k = []
        vv_v = []
        for k2,value_q in all_dict_7.items():
            if np.abs(np.subtract(values, value_q)).max() <= thred_v:
                ll_v.append(value_q)
                ll_k.append(k2)
            else:
                vv_v.append(value_q)
                vv_k.append(k2)
        if len(ll_v) != 0:
            index_coll_7[str(k)] = dict(zip(ll_k, ll_v))
            k = k+1
        if len(vv_v) != 0 and len(vv_v) != 1: 
            all_dict_7 = dict()
            all_dict_7 = dict(zip(vv_k, vv_v))
        else:
            break 
    if len(vv_v) != 0:       
        index_coll_7[str(k)] = dict(zip(vv_k, vv_v))
    puzzle_dict_7 = collections.OrderedDict()
    for i,v in index_coll_7.items():
            first_key = next(iter(v))
            puzzle_dict_7[i] = dict(zip([first_key],[v[first_key]]))
            continue
    corr_conc = len(puzzle_dict_3) + len(puzzle_dict_4)+len(puzzle_dict_5)+len(puzzle_dict_6)+len(puzzle_dict_7)
    print(f'{num}-----{thred_v}:  {corr_conc}')

5-----0.05:  345
5-----0.1:  188
5-----0.15:  118
5-----0.2:  75
5-----0.25:  62
5-----0.3:  43
5-----0.35:  34
5-----0.4:  25


In [9]:
index_coll_6

OrderedDict([('0',
              {'Ca1Cr1Mg1Na1O12Si4': array([[ 1.        ,  0.15310829, -0.29102586, -0.69014198, -0.32874092,
                       -0.16569028],
                      [ 0.15310829,  1.        ,  0.64281263, -0.21022304, -0.87943419,
                       -0.84833313],
                      [-0.29102586,  0.64281263,  1.        ,  0.42885081, -0.71831326,
                       -0.8262228 ],
                      [-0.69014198, -0.21022304,  0.42885081,  1.        ,  0.16030518,
                       -0.13239755],
                      [-0.32874092, -0.87943419, -0.71831326,  0.16030518,  1.        ,
                        0.87030508],
                      [-0.16569028, -0.84833313, -0.8262228 , -0.13239755,  0.87030508,
                        1.        ]]),
               'Al1Fe1K1Mg2O12Si3': array([[ 1.        ,  0.15305381, -0.33958547, -0.70914926, -0.28605638,
                       -0.17515962],
                      [ 0.15305381,  1.        ,  0.63995235,

In [7]:
formu

array(['Fe1Ru2Si1', 'Ag7Al2Si1', 'Cr1Ni1Si1', 'Al14Co5Si2', 'O1Si1Sr3',
       'Co3Er1Si2', 'Ca4N4Si1', 'Rh2Si3Y3', 'Fe31Si3Sm4', 'Be1O3Si1',
       'Ge3Nd10Si3', 'Fe2Si1Ti1', 'Pd1Si1Tb1', 'Ge1Mg4Si1', 'Se8Si2Ta11',
       'Fe15Nd2Si2', 'Ru2Si2U1', 'Na1Rb7Si8', 'Br3Si1Tb3', 'Al1Si3V12',
       'F6Na2Si1', 'Na6O7Si2', 'Ir2Si2Sm1', 'Au1Eu2Si3', 'Hf1Ni2Si2',
       'O7Si2Y2', 'Fe2Si3Zr3', 'O3Si1Th1', 'Hf2Ni3Si1', 'Mg1Nd1Si2',
       'Mn3Si12V17', 'Co1Ho1Si2', 'Ho2O5Si1', 'Cr3Si1Ti2', 'Ag1Dy1Si1',
       'Ni16Si7Zr6', 'Pd20Pr3Si6', 'Be2Co1Si1', 'Mn2Mo1Si1', 'Ba2Mg3Si4',
       'Al1Ca1Si1', 'Nd1Si2Zn2', 'Cd1P2Si1', 'La1Pt2Si2', 'Fe2Si2Sm1',
       'Nd1Ni9Si2', 'Ce1Mn2Si2', 'Al3Er2Si2', 'Os1Si3Th1', 'Fe1Si2Y2',
       'Nb3Ni2Si1', 'Nd5O13Si3', 'Fe3Ni3Si2', 'Fe5Gd2Si8', 'Au2Ni4Si7',
       'Fe1Si3Tm3', 'Ru2Si1Zr1', 'Li2Nd2Si3', 'C1Cl7Si3', 'Ag1Pr1Si2',
       'Fe1Si1Tc2', 'Fe1Pr1Si2', 'Fe1Hf1Si1', 'Ba8Ni3Si43', 'Mn2Si2Yb1',
       'Rh2Si2Y3', 'Ru2Si2Tb1', 'Ce1La1Si2', 'Ga1La4Si3', 'Ce3S2Si1',